In [1]:
import numpy as np
import eventlet
import tensorflow as tf
import socketio
import cv2
import eventlet.wsgi
from flask import Flask
from PIL import Image
import base64
from io import BytesIO
import os

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Image details

In [2]:
height = 32
width = 32
channel = 3

In [3]:
def readImage(imageName):
    global ImageDir
    return mpimg.imread(os.path.join(ImageDir, imageName.strip()))

In [4]:
def getInputValues(image):
    global height, width, channel
        
    images  = np.empty(shape =[1, height,width,channel], dtype=np.float32)
    
    _image= imageResize(image)
    
    images[0] = (_image/127.5) -1 
        
    return images
        
    

In [5]:
def imageResize(image):
    global height, width
    return cv2.resize(image, (width, height), cv2.INTER_AREA)

In [6]:
sess = tf.Session()
saver = tf.train.import_meta_graph("model/Model_15425395460637_____9.meta")
saver.restore(sess, tf.train.latest_checkpoint('model'))
graph = tf.get_default_graph()
    
Xplace = graph.get_tensor_by_name("X_Placeholder:0")
output = graph.get_tensor_by_name("dense_2/BiasAdd:0")
 

INFO:tensorflow:Restoring parameters from model\Model_15425395460637_____9


In [7]:
def predictValue(image):
    x = getInputValues(image)
    _yPred = sess.run(output,feed_dict = {Xplace:x})        
    return _yPred

# Server 

In [8]:
sio = socketio.Server()

In [9]:
app = Flask(__name__)

In [10]:
@sio.on('telemetry')
def telemetry(sid, data):    

    image = Image.open(BytesIO(base64.b64decode(data["image"])))
    try:
        image = np.asarray(image)
        temp = predictValue(image)
        steering_angle =float(temp[0])  
        print(steering_angle)
        
        send_control(steering_angle, 1)
    except Exception as e:
        print(e)

In [11]:
@sio.on('connect')
def connect(sid, environ):
    send_control(0, 0)
    print("connected")

In [12]:
def send_control(steering_angle, accelerator):  
    sio.emit(
        "steer",
        data={
            'steering_angle': steering_angle.__str__(),
            'accelerator': accelerator.__str__()
        },
        skip_sid=True)

In [ ]:
app = socketio.Middleware(sio, app) 

In [ ]:
eventlet.wsgi.server(eventlet.listen(('', 9876)), app)

(5644) wsgi starting up on http://0.0.0.0:9876
(5644) accepted ('127.0.0.1', 57640)


connected


127.0.0.1 - - [18/Nov/2018 17:09:02] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 757.422655


In [ ]:
sess.close()